In [1]:
pip install rdkit

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from rdkit import Chem

C:\Users\Владимир\AppData\Local\Temp\ipykernel_5952\219922904.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df_raw = pd.read_csv("test_data.csv")
df_raw.head()

,Unnamed: 0,Drug_ID,Drug,property
0,0,0,Cc1cc(=O)oc2ccc3c(c12)C1(C)OOC1(C)O3,1
1,1,1,CN(CC(O)CO)N=O,1
2,2,2,O=[N+]([O-])c1ccc(OC2CO2)cc1,1
3,3,3,Cn1cnc2cc([N+](=O)[O-])ccc21,1
4,4,4,CC[C@H](CO)NCCN[C@H](CC)CO,1


In [4]:
df_clean = df_raw.drop(["Unnamed: 0", "Drug_ID"], axis=1)

In [5]:
df_clean.head()

,Drug,property
0,Cc1cc(=O)oc2ccc3c(c12)C1(C)OOC1(C)O3,1
1,CN(CC(O)CO)N=O,1
2,O=[N+]([O-])c1ccc(OC2CO2)cc1,1
3,Cn1cnc2cc([N+](=O)[O-])ccc21,1
4,CC[C@H](CO)NCCN[C@H](CC)CO,1


In [6]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Drug      1221 non-null   object
 1   property  1221 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 19.2+ KB


In [7]:
from rdkit.Chem import Descriptors, MolFromSmiles

In [8]:
computed_descriptors = Chem.Descriptors.descList
for descriptor in computed_descriptors:
    name = descriptor[0]
    df_clean[name] = df_clean["Drug"].apply(lambda x: descriptor[1](MolFromSmiles(x)))

df_clean.head()

[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:13] WARNING: not removing hydrogen atom without neighbors
[20:53:14] WARNING: not removing hydrogen atom without neighbors
[20:53:14] WARNING: not removing hydrogen atom without neighbors
[20:53:14] WARNING: not removing hydrogen atom without neighbors
[20:53:14] WARNING: not removing hydrogen atom without neighbors
[20:53:14] WARNING: not r

,Drug,property,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,Cc1cc(=O)oc2ccc3c(c12)C1(C)OOC1(C)O3,1,11.453287,11.453287,0.358743,-0.827190,0.537391,31.526316,260.245,248.149,...,0,0,0,0,0,0,0,0,0,0
1,CN(CC(O)CO)N=O,1,9.639436,9.639436,0.069444,-0.881759,0.379944,12.777778,134.135,124.055,...,0,0,0,0,0,0,0,0,0,0
2,O=[N+]([O-])c1ccc(OC2CO2)cc1,1,10.286822,10.286822,0.059320,-0.447020,0.400523,19.538462,181.147,174.091,...,0,0,0,0,0,0,0,0,0,0
3,Cn1cnc2cc([N+](=O)[O-])ccc21,1,10.422392,10.422392,0.075185,-0.422500,0.489245,10.538462,177.163,170.107,...,0,0,0,0,0,0,0,0,0,0
4,CC[C@H](CO)NCCN[C@H](CC)CO,1,8.903975,8.903975,0.188201,0.188201,0.389671,15.428571,204.314,180.122,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_clean.to_csv("test_with_descriptors.csv")